# Cubic Hermite Splines 


## Unit interval $(0, 1)$
Given endpoints $\boldsymbol{p}(0) = p_0$ and $\boldsymbol{p}(1) = p_1$ and derivatives $\boldsymbol{p}^\prime(0) = m_0$ and $\boldsymbol{p}^\prime(1) = m_1$, the polynomial can be uniquely defined by: 

$\boldsymbol{p}(t) = h_{00}(t)\boldsymbol{p}_0 + h_{10}(t)\boldsymbol{m}_0 + h_{01}(t)\boldsymbol{p}_1 + h_{11}(t)\boldsymbol{m}_1$

where $h_{00}$, $h_{10}$, $h_{01}$, $h_{11}$ are Hermite basis functions:

| Basis&#x2001; | &#x2001; Expanded &#x2001; | &#x2001;&#x2001; Factorized  &#x2001; |
|---------------|--------------------|------------------------|
| $h_{00}(t)$   | $2t^3-3t^2+1$      | $(1 + 2 t) ( 1 - t)^2$ |
| $h_{10}(t)$   | $t^3-2t^2+t$       | $t (1 - t)^2$          |
| $h_{01}(t)$   | $-2t^3+3t^2$       | $t^2 (3 - 2 t)$        |
| $h_{11}(t)$   | $t^3-t^2$          | $t^2 (t - 1)$          |

and $t ∈ [0, 1]$.

The "factorized" column shows immediately, that $h_{10}$ and $h_{11}$ are zero at the boundaries.
You can further conclude that $h_{01}$ and $h_{11}$ have a zero of multiplicity 2 at 0 and $h_{00}$ and $h_{10}$ have such a zero at 1, thus they have slope 0 at those boundaries.

## Interpolation on an arbitrary interval.

Interpolating $x$ in an arbitrary interval $(x_0, x_1)$ is done by mapping the latter to $t ∈ [0,1]$ through an affine change of variable: $t = \frac{x - x_0}{x_1 - x_0}$.

Not that this requires derivative scaling: $\frac{dp}{dt} = \frac{dp}{dx} \left(x_1 - x_0\right)$ and $\frac{dp}{dx} = \frac{dp}{dt} \left(x_1 - x_0\right)^{-1}$.

## Implementation 
The polynomial is most effeciently evaluated in [horner form](https://en.wikipedia.org/wiki/Horner%27s_method), and can be further specialized for the common "interpolate to zero" case.

In [4]:
import sympy

def poly(coeffs, var):
    return sympy.Poly.from_list(coeffs, var).as_expr()

def horner_poly(expr, var):
    from sympy.polys.polyfuncs import horner
    return horner(sympy.Poly(expr), var)

In [5]:
t, p0, p1, dp0, dp1 = sympy.symbols("t, p0, p1, dp0, dp1")
h_00 = poly((2, -3, 0, 1), t)
h_10 = poly((1, -2, 1, 0), t)
h_01 = poly((-2, 3, 0, 0), t)
h_11 = poly((1, -1, 0, 0), t)

p = h_00 * p0 + h_10 * dp0 + h_01 * p1 + h_11 * dp1

In [6]:
cubic_hermite_forms = f'''
def interpolate_t(t, p0, dp0, p1,  dp1):
    """Cubic hermite interpolation of p on t in [0, 1]."""
    return {horner_poly(p, t)}
    
def interpolate_dt(t, p0, dp0, p1, dp1):
    """Cubic hermite interpolation of dp/dt on t in [0, 1]."""
    return {horner_poly(p.diff(t), t)}
    
def interpolate_to_zero_t(t, p0, dp0):
    """Cubic hermite interpolation of p on t in [0, 1] to (p1, dp1) == 0."""
    return {horner_poly(p.subs([(p1, 0), (dp1, 0)]), t)}
    
def interpolate_to_zero_dt(t, p0, dp0):
    """Cubic hermite interpolation of dp/dt on t in [0, 1] to (p1, dp1) == 0."""
    return {horner_poly(p.subs([(p1, 0), (dp1, 0)]), t)}
'''

display({"text/markdown":f"""
```python
{cubic_hermite_forms}
```
"""},
raw=True)

exec(cubic_hermite_forms)


```python

def interpolate_t(t, p0, dp0, p1,  dp1):
    """Cubic hermite interpolation of p on t in [0, 1]."""
    return p0 + t*(dp0 + t*(-2*dp0 - dp1 - 3*p0 + 3*p1 + t*(dp0 + dp1 + 2*p0 - 2*p1)))
    
def interpolate_dt(t, p0, dp0, p1, dp1):
    """Cubic hermite interpolation of dp/dt on t in [0, 1]."""
    return dp0 + t*(-4*dp0 - 2*dp1 - 6*p0 + 6*p1 + t*(3*dp0 + 3*dp1 + 6*p0 - 6*p1))
    
def interpolate_to_zero_t(t, p0, dp0):
    """Cubic hermite interpolation of p on t in [0, 1] to (p1, dp1) == 0."""
    return p0 + t*(dp0 + t*(-2*dp0 - 3*p0 + t*(dp0 + 2*p0)))
    
def interpolate_to_zero_dt(t, p0, dp0):
    """Cubic hermite interpolation of dp/dt on t in [0, 1] to (p1, dp1) == 0."""
    return p0 + t*(dp0 + t*(-2*dp0 - 3*p0 + t*(dp0 + 2*p0)))

```
